# FLOWCELL | OPERATION

[2024-05-09] Notez ici le suivi des opérations en commençant par modifier en titre les noms de la _flowcell_ (_e.g._.: "230711_A00516_0433_BHKVMJDMXY") et de l'exérience ("Seq_S2_PRAG_20230711"). Ces informations sont normalement communiqués dans un courriel du laboratoire CQGC, mais peuvent aussi être récupérées depuis BSSH (sous l'onglet "_Runs_"). **_N.B._** Le format `ipynb` (Jupyter Notebook) est utilisé car du code peut y être exécuté, mais un simple format `txt` ou `md` peut aussi bien servir.

0. Mise-en-place de l'environnement de travail
1. Collecter les métriques
2. Téléverser les FASTQs sur BaseSpace
3. Créer les cas sur Emedgene 
    1. Générer le fichier "emg_batch_manifest.csv"
    2. Créer les cas dans Emedgene (à l'aide du fichier "emg_batch_manifest.csv") 
    3. (**TODO**) Ajouter les participants _via_ l'API
4. Archiver les résultats


In [ ]:
## 0. Mise en place de l'environnement de travail
# ssh ${USER}@10.128.80.26 # Renseigner la valeur de ${FC}

screen -S prag
conda activate CQGC-utils

export FC=""
a=($(echo ${FC} | tr '_' '\n'))
export FC_SHORT="${a[1]}_${a[2]}"
export BASEDIR="/mnt/spxp-app02/staging/hiseq_raw/${a[1]}"
export WORKDIR="/mnt/spxp-app02/staging2/dragen"


In [ ]:
## 1. Collecter les metriques

cd ${WORKDIR}
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_dragen_metrics.py ${FC}

Récupérer les métriques et le fichier "samples_list.csv" générés par la commande ci-dessus. 

```PowerShell:
Set-Location D:\HSJ\Projects\PRAGMatIQ\Runs\LH00336_0073
scp hienle@10.128.80.26:/mnt/spxp-app02/staging2/dragen/LH00336_0073/* .
```
Envoyer les fichiers `${FC_SHORT}_metrics.csv` et `${FC_SHORT}_metrics.html` dans un courriel aux personnes responsables:

- "Dr Jacques Michaud (HSJ)" <jacques.michaud.med@ssss.gouv.qc.ca>
- "Camille Varin-Tremblay (HSJ)" <camille.varin-tremblay.hsj@ssss.gouv.qc.ca>
- "Rene Allard (HSJ)" <rene.allard.hsj@ssss.gouv.qc.ca>

En attendant l'approbation, procéder au transfert des FASTQs sur BaseSpace


In [ ]:
## 2. Upload FASTQ sur BaseSpace

cd ${WORKDIR}/${FC_SHORT}
python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_upload_fastqs.py

In [ ]:
## 3. Générer le fichier `emg_batch_manifest.csv`

python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_make_batch_from_nanuq.py ${FC_SHORT}`

Supprimer manuellement du fichier `emg_batch_manifest.csv` les échantillons qui n'ont pas été approuvés par les responsables.
Se connecter à Emedgene et soumettre le fichier CSV à la cration des cas par lot.

In [ ]:
## 4. Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done